In [20]:
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn

In [1]:
#things to note: The sample rate is quite high. If it becomes computationally expensive
#we can always dumb down the sampling rate

In [23]:
df_full = pd.read_csv("./20240229_2308.csv", header=None, engine='python')

# The maximum length of the dataset is the number of rows
max_length = df_full.shape[0]

# Print the maximum length
print(f"The maximum length of the dataset is: {max_length}")

# Now, you can use this information to set length_read appropriately
length_read = min(12546000, max_length)  # for example, to limit it to 100000 or the maximum length available

# Proceed with loading the desired portion of the dataset
df = pd.read_csv("./20240229_2308.csv", header=None).iloc[1:length_read, :6]
df.columns = ["l1","l2","l3","r1","r2","r3"]
df.head()

#the maximum length is 12546118

The maximum length of the dataset is: 12546118


C:\Users\yueze\AppData\Local\Temp\ipykernel_26244\2852126022.py:13: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./20240229_2308.csv", header=None).iloc[1:length_read, :6]


,l1,l2,l3,r1,r2,r3
1,-2.390720118768513,-0.5347981965169311,-0.16361381206661463,0.9987793918699026,-0.13430978171527386,-2.253967977128923
2,-2.400488128885627,-0.5347981965169311,-0.16361381206661463,1.0085474019870162,-0.15384580194950104,-2.2637359872460365
3,-2.400488128885627,-0.5347981965169311,-0.16361381206661463,1.0085474019870162,-0.15384580194950104,-2.2637359872460365
4,-2.390720118768513,-0.5347981965169311,-0.16361381206661463,0.9987793918699026,-0.13430978171527386,-2.2637359872460365
5,-2.400488128885627,-0.5347981965169311,-0.17338182218372822,1.0085474019870162,-0.13430978171527386,-2.2637359872460365


In [29]:
print (max_length)
eval_interval = 100

12546118


In [25]:
targets = ["r1", "r2", "r3"]

In [26]:
x = df.drop(columns=targets)
y = df[targets]
split_idx = int(len(df) * 0.9)

# Split into training and testing sets
x_train = x.iloc[:split_idx]
y_train = y.iloc[:split_idx]
x_test = x.iloc[split_idx:]
y_test = y.iloc[split_idx:]

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim



input_feature_dim = 3  # Each input element is a 1x3 vector
embed_size = 64
target_dim = 3
block_size = 8
num_heads = 16
max_iters = 1000
batch_size = 10



def get_batch3(split):
    # Select the correct data split
    if split == 'train':
        a, b, max_index = x_train, y_train, int(length_read * 0.9) - block_size - 1
    else:  # split == 'test'
        a, b, max_index = x_test, y_test, length_read - (int(length_read * 0.9) + block_size + 1)
    
    # Generate random indices for batch selection, ensuring they're within bounds
    ix = torch.randint(0, max_index, (batch_size,))

    # Initialize lists to hold the batches
    x_batch = []
    y_batch = []

    for i in ix:
        try:
            # Extract the sequence from 'a' and the corresponding target from 'b'
            seq = torch.tensor(a.iloc[i.item():i.item() + block_size].astype(np.float32).values, dtype=torch.float32)
            target = torch.tensor(b.iloc[i.item() + block_size].astype(np.float32).values, dtype=torch.float32)
            x_batch.append(seq)
            y_batch.append(target)
        except IndexError as e:
            print(f"IndexError for index {i.item()}: {str(e)}")
            print(f"Attempting to access index [{i.item()}:{i.item() + block_size}] in 'a' with shape {a.shape}")
            print(f"Attempting to access index {i.item() + block_size} in 'b' with shape {b.shape}")
            # Optionally, break or continue depending on desired behavior on error
            break  # or continue

    if not x_batch or not y_batch:
        print("Error: Batch could not be created due to index issues.")
        return None, None
    
    # Stack the collected sequences and targets into tensors
    xstack = torch.stack(x_batch)
    ystack = torch.stack(y_batch)

    return xstack, ystack
class SelfAttention(nn.Module):
    def __init__(self, embed_size):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        
        self.keys = nn.Linear(embed_size, embed_size, bias=False)
        self.queries = nn.Linear(embed_size, embed_size, bias=False)
        self.values = nn.Linear(embed_size, embed_size, bias=False)
        
    def forward(self, x):
        K = self.keys(x)
        Q = self.queries(x)
        V = self.values(x)
        
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.embed_size ** 0.5
        attention = torch.softmax(attention_scores, dim=-1)
        
        attended = torch.matmul(attention, V)
        return attended

class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.embed_size = embed_size
        self.num_heads = num_heads
        
        assert embed_size % num_heads == 0
        
        self.head_dim = embed_size // num_heads
        
        self.keys = nn.Linear(embed_size, embed_size, bias=False)
        self.queries = nn.Linear(embed_size, embed_size, bias=False)
        self.values = nn.Linear(embed_size, embed_size, bias=False)
        
        self.fc_out = nn.Linear(embed_size, embed_size)
    
    def forward(self, x):
        batch_size, seq_length, _ = x.shape
        
        keys = self.keys(x).view(batch_size, seq_length, self.num_heads, self.head_dim)
        queries = self.queries(x).view(batch_size, seq_length, self.num_heads, self.head_dim)
        values = self.values(x).view(batch_size, seq_length, self.num_heads, self.head_dim)
        
        attention_scores = torch.einsum("bnqh,bnkh->bnqk", [queries, keys]) / (self.head_dim ** 0.5)
        attention = torch.softmax(attention_scores, dim=-1)
        
        attended = torch.einsum("bnqk,bnkv->bnqv", [attention, values]).reshape(batch_size, seq_length, self.embed_size)
        
        output = self.fc_out(attended)
        return output
        
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch3(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
    
class EncoderWithFullyConnected(nn.Module):
    def __init__(self, input_feature_dim, embed_size, seq_length, num_heads):
        super(EncoderWithFullyConnected, self).__init__()
        self.input_fc = nn.Linear(input_feature_dim, embed_size)
        self.multi_head_attention = MultiHeadAttention(embed_size, num_heads)
        self.positional_encoding = nn.Parameter(torch.randn(1, seq_length, embed_size))
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.input_fc(x)) + self.positional_encoding
        x = self.multi_head_attention(x)
        return x

class EncoderDecoderModelWithMultiHeadAttention(nn.Module):
    def __init__(self, input_feature_dim, embed_size, target_dim, seq_length, num_heads):
        super(EncoderDecoderModelWithMultiHeadAttention, self).__init__()
        self.encoder = EncoderWithFullyConnected(input_feature_dim, embed_size, seq_length, num_heads)
        self.decoder = nn.Sequential(
            nn.Linear(embed_size, target_dim),
        )

    def forward(self, x,targets):
        encoded = self.encoder(x)
        encoded_pooled = torch.mean(encoded, dim=1)
        decoded = self.decoder(encoded_pooled)

        loss = criterion(decoded, targets)

        return decoded,loss
        

# Example usage
'''
if __name__ == "__main__":
    input_feature_dim = 3  # Each input element is a 1x3 vector
    embed_size = 32
    target_dim = 3
    seq_length = 6
    num_heads = 8

    model = EncoderDecoderModelWithMultiHeadAttention(input_feature_dim, embed_size, target_dim, seq_length, num_heads)
    
    # Example input (batch_size, seq_length, input_feature_dim)
    x = torch.rand(4, seq_length, input_feature_dim)  # Batch size of 4
    targets = torch.rand(4, target_dim)  # Corresponding targets

    # Forward pass
    predictions = model(x)
    
    # Example training components
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Compute loss
    loss = criterion(predictions, targets)
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print("Loss:", loss.item())
'''

'\nif __name__ == "__main__":\n    input_feature_dim = 3  # Each input element is a 1x3 vector\n    embed_size = 32\n    target_dim = 3\n    seq_length = 6\n    num_heads = 8\n\n    model = EncoderDecoderModelWithMultiHeadAttention(input_feature_dim, embed_size, target_dim, seq_length, num_heads)\n    \n    # Example input (batch_size, seq_length, input_feature_dim)\n    x = torch.rand(4, seq_length, input_feature_dim)  # Batch size of 4\n    targets = torch.rand(4, target_dim)  # Corresponding targets\n\n    # Forward pass\n    predictions = model(x)\n    \n    # Example training components\n    criterion = nn.MSELoss()\n    optimizer = optim.Adam(model.parameters(), lr=0.001)\n    \n    # Compute loss\n    loss = criterion(predictions, targets)\n    \n    # Backpropagation\n    optimizer.zero_grad()\n    loss.backward()\n    optimizer.step()\n    \n    print("Loss:", loss.item())\n'

In [33]:
if __name__ == "__main__":
    model = EncoderDecoderModelWithMultiHeadAttention(input_feature_dim, embed_size, target_dim, block_size, num_heads)
    
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    for iter in range(max_iters):
    
        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
        # sample a batch of data
        xb, yb = get_batch3('train')
        predictions,loss = model(xb,yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
    print("Loss:", loss.item())

step 0: train loss 1.8066, val loss 1.8866
step 100: train loss 0.0430, val loss 0.5770
step 200: train loss 0.0025, val loss 0.0094
step 300: train loss 0.0011, val loss 0.0129
step 400: train loss 0.0009, val loss 0.0120
step 500: train loss 0.0011, val loss 0.0146
step 600: train loss 0.0009, val loss 0.0142
step 700: train loss 0.0008, val loss 0.0124
step 800: train loss 0.0011, val loss 0.0086
step 900: train loss 0.0008, val loss 0.0119
step 999: train loss 0.0008, val loss 0.0090
Loss: 0.0003775508375838399


In [36]:
eval_iters = 200
block_size = 1000

In [39]:
X_last_20 = x_test[-block_size-1:-1]
y_last_20_actual = y_test[-1:]
X_last_20_tensor = torch.tensor(X_last_20.astype(np.float32).values, dtype=torch.float32)
y_last_20_actual_tensor = torch.tensor(y_last_20_actual.astype(np.float32).values, dtype=torch.float32)
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Do not compute gradients
    y_last_20_pred = model(X_last_20_tensor,y_last_20_actual_tensor)
print("Predictions:", y_last_20_pred)
print ("actual:",y_last_20_actual_tensor)

Predictions: (tensor([[ 1.6832,  1.5022, -0.9166]]), tensor(0.0135))
actual: tensor([[ 1.8486,  1.3895, -0.8962]])


In [40]:
print(y_test.shape)

(1254600, 3)


In [ ]:
predictions=[]
for i in range(len(x_test) - 1001):
    if i + 1000 < len(y_test):
        y_window = y_test.iloc[i+1000]  # Use .iloc for safe access
        # Assuming x_test and y_test are pandas DataFrames or Series and you need their values as tensors
        seq = torch.tensor(x_test.iloc[i:i + block_size].astype(np.float32).values, dtype=torch.float32)
        target = torch.tensor(y_test.iloc[i + block_size].astype(np.float32).values, dtype=torch.float32)
    else:
        # Handle the case where i+100 exceeds the bounds of y_test, if necessary
        break
    with torch.no_grad():  # Do not compute gradients
        prediction,loss = model(x_window_tensor,y_window_tensor)  # Predict the next value based on the window
    predictions.append(prediction)

# The actual values that correspond to the predictions
# Since we predict the next value after each window, we start collecting actuals from the 101st value
actuals = y_test[1000:].numpy()

# Calculate Pearson's correlation coefficientc
correlation_coef, p_value = pearsonr(predictions, actuals)

In [ ]:
"""

print(x_train[:10])
#Here we want to describe each line into its own token. 
torch.manual_seed(1337)
batch_size = 4 #how many sequences we use in parallel
block_size = 8

def get_batch(split):
    # Select the correct data split
    a, b, length_read2 = (x_train, y_train, int(length_read * 0.9)) if split == 'train' else (x_test, y_test, int(length_read * 0.1))
    # Random indices for batch selection
    ix = torch.randint(length_read2 - block_size, (batch_size,))

    # Generate batches
    xstack = torch.stack([torch.tensor(a.iloc[i.item():i.item() + block_size].astype(np.float64).values, dtype=torch.double) for i in ix])
    # Assuming the target is structured similarly to the input and requires similar reshaping
    ystack = torch.stack([torch.tensor(b.iloc[i.item()+block_size+1].astype(np.float64).values, dtype=torch.double) for i in ix])

    return xstack, ystack
xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('-------')

for b in range(batch_size):
    context = xb[b,]
    target = yb[b,]
    print(f"When input is {context.tolist()} the target: {target}\n")blocksize = 1000

for t in range(blocksize):
    context = x_train.iloc[:t+1].astype(np.float64)  # Selects rows up to index t from x
    target = y_train.iloc[t].astype(np.float64)  # Selects the row at index t from y

    # Flatten the context and target
    flattened_context = torch.tensor(context.values,dtype=torch.float64)
    flattened_target = torch.tensor(target.values,dtype=torch.float64)  # Assuming 'target' is a DataFrame. If 'target' is a Series, it's already 1-D.
    if t < 10:
    # Print the flattened arrays
        print(f"when the input is \n{flattened_context} \nthe target:\n {flattened_target}")

def encode(x,y):
    context = x.iloc[:t+1].astype(np.float64)  # Selects rows up to index t from x
    target = y.iloc[t].astype(np.float64)  # Selects the row at index t from y

    # Flatten the context and target
    flattened_context = torch.tensor(context.values.flatten(),dtype=torch.float64)
    flattened_target = torch.tensor(target.values.flatten(),dtype=torch.float64)  # Assuming 'target' is a DataFrame. If 'target' is a Series, it's already 1-D.
    return flattened_context,flattened_target

blocksize = 1000

for t in range(blocksize):
    context = x_train.iloc[:t+1].astype(np.float64)  # Selects rows up to index t from x
    target = y_train.iloc[t].astype(np.float64)  # Selects the row at index t from y

    # Flatten the context and target
    flattened_context = torch.tensor(context.values,dtype=torch.float64)
    flattened_target = torch.tensor(target.values,dtype=torch.float64)  # Assuming 'target' is a DataFrame. If 'target' is a Series, it's already 1-D.
    if t < 10:
    # Print the flattened arrays
        print(f"when the input is \n{flattened_context} \nthe target:\n {flattened_target}")

def encode(x,y):
    context = x.iloc[:t+1].astype(np.float64)  # Selects rows up to index t from x
    target = y.iloc[t].astype(np.float64)  # Selects the row at index t from y

    # Flatten the context and target
    flattened_context = torch.tensor(context.values.flatten(),dtype=torch.float64)
    flattened_target = torch.tensor(target.values.flatten(),dtype=torch.float64)  # Assuming 'target' is a DataFrame. If 'target' is a Series, it's already 1-D.
    return flattened_context,flattened_target
    
torch.manual_seed(1337)
batch_size = 4 #how many sequences we use in parallel
block_size = 8
def get_batch(split):
    #generate a small batch of data of inputs x and targets y
    a, b, length_read2 = (x_train, y_train,length_read*0.9) if split == 'train' else (x_test, y_test,length_read*0.1)
    ix = torch.randint(int(length_read2)-block_size, (batch_size, ))

    # Flatten the context and target
    xstack = torch.stack([torch.tensor(a.iloc[i.item():i.item()+block_size].astype(np.float64).values.flatten(), dtype=torch.double) for i in ix])
    ystack = torch.stack([torch.tensor(b.iloc[i.item()+block_size+1].astype(np.float64).values.flatten(), dtype=torch.double) for i in ix])

    return xstack,ystack
xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('-------')

for b in range(batch_size):
    context = xb[b,]
    target = yb[b,]
    print(f"When input is {context.tolist()} the target: {target}\n")
'''